In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.3 MB/s 
     |████████████████████████████████| 88 kB 6.2 MB/s 
     |████████████████████████████████| 636 kB 48.4 MB/s 
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 5.2 MB 17.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 5.5 MB 44.6 MB/s 
     |████████████████████████████████| 1.3 MB 63.5 MB/s 
     |████████████████████████████████| 182 kB 65.0 MB/s 
     |████████████████████████████████| 7.6 MB 54.8 MB/s 
     |████████████████████████████████| 1.1 MB 65.1 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=2340716 sha256=50b43356f2be99996da907e100a60d4129e224c34205eba3ed654c882a0d5bdf
  Stored in directory: /root/.cache

In [2]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 10.08 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 761 kB/s 
     |████████████████████████████████| 465 kB 64.7 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-11-27 12:48:50--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (b

# 데이터

학습을 위한 데이터가 필요합니다. 여기서는 하나의 라인(line)에 하나의 문서로 구성된 파일이 필요한데요. 우선, 여러분들의 데이터가 없다면 여기서 준비한 파일로 실습을 해봅시다.

In [4]:
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
captionTable = pd.read_csv('/content/drive/MyDrive/kusw_second_project/captionTable.csv')

In [7]:
video_id_list = list(set(captionTable['video_id']))

In [8]:
def line2doc(video_id_idx):
  doc_df = captionTable[captionTable['video_id'] == video_id_list[video_id_idx]]
  return ' '.join(list(doc_df['captions'][1:]))

In [9]:
doc_list = list(map(line2doc, range(len(video_id_list))))

In [10]:
len(doc_list)

918

# 필요한 것들을 임포트

In [13]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic

# 전처리

# Mecab과 SBERT를 이용한 Bertopic

In [14]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [35]:
class CustomTokenizer_nv:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        # sent = sent[:1000000]
        word_tokens = self.tagger.pos(sent)
        result = [word[0] for word in word_tokens if (word[1].startswith('N') or word[1].startswith('VA')) and len(word[0]) > 1]
        return result

In [36]:
custom_tokenizer = CustomTokenizer_nv(Mecab())

In [37]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [38]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=50,
                 top_n_words=10,
                 calculate_probabilities=True)

In [39]:
topics, probs = model.fit_transform(doc_list)

In [40]:
model.visualize_topics()

In [41]:
model.visualize_distribution(probs[0])

In [42]:
for i in range(0, 50):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('우리', 0.04117245758639976), ('유전자', 0.03922399270631661), ('선택', 0.037438567736181205), ('연결망', 0.03335263593106347), ('나무', 0.02936217190075508), ('사람', 0.02396308442474833), ('사회', 0.021509676531395034), ('나라', 0.02133679547070368), ('아이', 0.020825393854787457), ('여러분', 0.020779604391239413)]
1 번째 토픽 : [('전쟁', 0.08055232317779389), ('러시아', 0.07252994982569848), ('우크라이나', 0.03953879916677169), ('푸틴', 0.03837589330892547), ('미국', 0.035006014622529724), ('마약', 0.03441507067940801), ('유럽', 0.029821254943662393), ('국가', 0.026846484447385417), ('사람', 0.02588957697455963), ('우크라', 0.02548162722868467)]
2 번째 토픽 : [('의사', 0.07467559282470315), ('파지', 0.05603283718075167), ('환자', 0.05324328816360085), ('우리', 0.03719453199937223), ('박테리아', 0.03454409429100614), ('치료', 0.030610869179320682), ('사람', 0.03015629486556434), ('그녀', 0.029131914857423294), ('실수', 0.02801529540526606), ('발생', 0.02731217527643839)]
3 번째 토픽 : [('돌고래', 0.04030722136132692), ('우리', 0.03973256019211413), ('바다', 0